# Logistic Regression

In [49]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.svm import SVC
import sklearn as sk
import seaborn as sns 
import pandas as pd
import numpy as np
import warnings

In [60]:
#given a dataset it returns a list will all the values of the targets (hypo = 0 and Norm = 0)
def labels(df):
    target = []
    for c in df.columns:
        if "Hypoxia" in c.replace('\"', '').split("_") or "Hypo" in c.split("_"):
            target.append(0.0)
        elif "Normoxia" in c.replace('\"', '').split("_") or "Norm" in c.split("_"):
            target.append(1.0)
        else:
            
            print(c)
            raise ValueError("Cell cannot be categorized")
    return target
        

In [56]:
#-----SmartSeq-----#
HCC_s_Train = pd.read_csv("processed_data\HCC1806_SmartS_Filtered_Standardized-Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)
MCF_s_Train = pd.read_csv("processed_data\MCF7_SmartS_Filtered_Standardized-Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)
HCC_s_Test = pd.read_csv("processed_data\HCC1806_SmartS_Filtered_Standardized-Normalised_3000_Data_test.txt", delimiter="\ ",engine='python',index_col=0)
MCF_s_Test = pd.read_csv("processed_data\MCF7_SmartS_Filtered_Standardized-Normalised_3000_Data_test.txt", delimiter="\ ",engine='python',index_col=0)

#-----DropSeq-----#
HCC_d = pd.read_csv("raw_data_DropSeq\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)
MCF_d = pd.read_csv("raw_data_DropSeq\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ",engine='python',index_col=0)


In [66]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def logi_model(df, y):
    pipe = make_pipeline(StandardScaler(), LogisticRegression())
    model = pipe.fit(df.T, y)
    return model

#-----SmartSeq-----#
HCC_s_logi_model = logi_model(HCC_s_Train, labels(HCC_s_Train))
MCF_s_logi_model = logi_model(MCF_s_Train, labels(MCF_s_Train))

#-----DropSeq-----#
HCC_d_train, HCC_d_test, y_HCC_d_train, y_HCC_d_test = train_test_split(HCC_d.T, labels(HCC_d))
MCF_d_train, MCF_d_test, y_MCF_d_train, y_MCF_d_test = train_test_split(MCF_d.T, labels(MCF_d))
HCC_d_logi_model = logi_model(HCC_d_train.T, y_HCC_d_train)
MCF_d_logi_model = logi_model(MCF_d_train.T, y_MCF_d_train)


hi


# Perfromance Measures

In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
warnings.simplefilter('ignore')

def cv_score(df, model):
    scaler = StandardScaler()
    return cross_val_score(model, scaler.fit_transform(df.T), labels(df), cv=KFold(shuffle=True, n_splits=5), scoring="accuracy")

#-----SmartSeq-----#
print("Cross validation score: ", cv_score(HCC_s_Train, HCC_s_logi_model))
print("Cross validation score: ", cv_score(MCF_s_Train, MCF_s_logi_model))

#-----DropSeq-----#
print("Cross validation score: ", cv_score(HCC_d_train.T, HCC_d_logi_model))
print("Cross validation score: ", cv_score(MCF_d_train.T, MCF_d_logi_model))

Cross validation score:  [0.97297297 0.91891892 1.         0.97222222 0.97222222]
Cross validation score:  [1. 1. 1. 1. 1.]
Cross validation score:  [0.92646391 0.92824705 0.92960945 0.93142598 0.92597639]
Cross validation score:  [0.9648582  0.96547472 0.96023428 0.95961776 0.95837188]


In [72]:
from sklearn.metrics import confusion_matrix

def pred_accuracy(df, model, title=""):
    print(title)
    print("Score: ", model.score(df.T, y=labels(df)))
    print("confusion_matrix: \n", confusion_matrix(model.predict(df.T), labels(df)), "\n")

#-----SmartSeq-----#
pred_accuracy(HCC_s_Test, HCC_s_logi_model, "SmartSeq HCC:")
pred_accuracy(MCF_s_Test, MCF_s_logi_model, "SmartSeq MCF:")

#-----DropSeq-----#
pred_accuracy(HCC_d_test.T, HCC_d_logi_model, "DropSeq HCC:")
pred_accuracy(MCF_d_test.T, MCF_d_logi_model, "DropSeq MCF:")


SmartSeq HCC:
Score:  0.9555555555555556
confusion_matrix: 
 [[18  2]
 [ 0 25]] 

SmartSeq MCF:
Score:  0.9838709677419355
confusion_matrix: 
 [[29  1]
 [ 0 32]] 


Score:  0.9234540997003541
confusion_matrix: 
 [[2073  109]
 [ 172 1317]] 


Score:  0.9659700388385426
confusion_matrix: 
 [[2164  110]
 [  74 3059]] 

